# Medicare Prime 

### Early Stage Skin Disese Detection

- **Objective**: Predictions of skin diseases in their early stages is today's need. For achieving this, We have multi-layered CNN-model with specifications as it was decided with random guesses and training experience.

#### Imports

In [ ]:
import pandas as pd
import numpy as np
#import keras
#from keras import backend as K

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

## **ABSTRACT :**<br>

<div align="center">
<img src=
"https://user-images.githubusercontent.com/78396437/153776325-4abbaf74-0860-4bf0-92cc-79f310a82603.png" 
         alt="logo" 
         align="center" 
         width="400">
</div>

-  **Today skin diseases are one of the most common diseases, whose number has been increasing day by day. Mostly, people neglect skin diseases and treatment procedures.This neglectance at the initial stages proves pernicious. But even if people consult physicians, it is quite difficult for them to precisely detect skin diseases with high accuracy and precision. Especially when it comes to the diseases like Melanoma, Actinic Keratoses, Basal cell carcinoma, Benign keratosis, Dermatofibroma, Melanocytic nevi, etc.**

-  **Predictions of skin diseases in their early stages is today's need. For this, we have built a multi-layered Early Skin Disease Detection MobileNet CNN model(proposed as a deep learning model by Andrew Howard) having 91 layers in it.**

-  **We have used this model to classify images in following 9 classes:**

    - Actinic Keratoses
    - Basal cell carcinoma
    - Benign keratosis
    - Dermatofibroma
    - Melanoma
    - Melanocytic nevi
    - Vascular skin lesions
    - Normal Human Skin Patch
    - Un-Zoomed human images

- **The dataset for above 9 classes is prepared from following sources:**

    1. HAM-10000 for:

        - Actinic Keratoses
        - Basal cell carcinoma
        - Benign keratosis
        - Dermatofibroma
        - Melanoma
        - Melanocytic nevi
        - Vascular skin lesions

    2. ISIC 2019 for:

        - Actinic-Keratoses
        - Basal cell carcinoma
        - Benign keratosis
        - Dermatofibroma
        - Melanoma
        - Vascular skin lesions

    3. UTK-Face: For Un-Zoomed human images

    4. Normal Human Skin Patch images were custom datasets built using Un-Zoomed human images. We cropped these images around forehead, cheeks and throat of human babies.
 


In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

##### **THIS DATA SET CAN BE FOUND**  [HERE](https://drive.google.com/drive/folders/1YjFmWfT0JnMJfDHxFWBdtHDjTdRckPtN?usp=sharing)



In [ ]:
import os
os.chdir("/content/drive/My Drive")

In [ ]:
list_train = []
list_train.append(len(os.listdir('Medicare_Prime/base_dir/train_dir/akiec')))
list_train.append(len(os.listdir('Medicare_Prime/base_dir/train_dir/bcc')))
list_train.append(len(os.listdir('Medicare_Prime/base_dir/train_dir/bkl')))
list_train.append(len(os.listdir('Medicare_Prime/base_dir/train_dir/df')))
list_train.append(len(os.listdir('Medicare_Prime/base_dir/train_dir/face')))
list_train.append(len(os.listdir('Medicare_Prime/base_dir/train_dir/mel')))
list_train.append(len(os.listdir('Medicare_Prime/base_dir/train_dir/normal')))
list_train.append(len(os.listdir('Medicare_Prime/base_dir/train_dir/nv')))
list_train.append(len(os.listdir('Medicare_Prime/base_dir/train_dir/vasc')))

sum(list_train)

In [ ]:
list_valid = []
list_valid.append(len(os.listdir('Medicare_Prime/base_dir/val_dir/akiec')))
list_valid.append(len(os.listdir('Medicare_Prime/base_dir/val_dir/bcc')))
list_valid.append(len(os.listdir('Medicare_Prime/base_dir/val_dir/bkl')))
list_valid.append(len(os.listdir('Medicare_Prime/base_dir/val_dir/df')))
list_train.append(len(os.listdir('Medicare_Prime/base_dir/val_dir/face')))
list_valid.append(len(os.listdir('Medicare_Prime/base_dir/val_dir/mel')))
list_train.append(len(os.listdir('Medicare_Prime/base_dir/val_dir/normal')))
list_valid.append(len(os.listdir('Medicare_Prime/base_dir/val_dir/nv')))
list_valid.append(len(os.listdir('Medicare_Prime/base_dir/val_dir/vasc')))

list_valid

### **DESCRIPTION OF DISEASES**

<div align="center">
<img src=
"https://user-images.githubusercontent.com/78396437/153776727-7e5e1e2a-d3fc-49e5-b414-397ebda66d05.png" 
         alt="logo" 
         align="center" 
         width="500">
</div>

- **nv**<br>
    - Melanocytic nevi are benign neoplasms of melanocytes and appear in a myriad of variants, which all are included in our series. The variants may differ significantly from a dermatoscopic point of view.<br>

 
- **mel**<br>
    - Melanoma is a malignant neoplasm derived from melanocytes that may appear in different variants. If excised in an early stage it can be cured by simple surgical excision. Melanomas can be invasive or non-invasive (in situ). We included all variants of melanoma including melanoma in situ, but did exclude non-pigmented, subungual, ocular or mucosal melanoma.<br>*
 
 
- **bkl**<br>
    - "Benign keratosis" is a generic class that includes seborrheic ker- atoses ("senile wart"), solar lentigo - which can be regarded a flat variant of seborrheic keratosis - and lichen-planus like keratoses (LPLK), which corresponds to a seborrheic keratosis or a solar lentigo with inflammation and regression. The three subgroups may look different dermatoscop- ically, but we grouped them together because they are similar biologically and often reported under the same generic term histopathologically. From a dermatoscopic view, lichen planus-like keratoses are especially challeng- ing because they can show morphologic features mimicking melanoma and are often biopsied or excised for diagnostic reasons.<br>

- **bcc**<br>
    - Basal cell carcinoma is a common variant of epithelial skin cancer that rarely metastasizes but grows destructively if untreated. It appears in different morphologic variants (flat, nodular, pigmented, cystic, etc) , which are all included in this set.<br>
 
- **akiec**<br>
    - Actinic Keratoses (Solar Keratoses) and intraepithelial Carcinoma (Bowen’s disease) are common non-invasive, variants of squamous cell car- cinoma that can be treated locally without surgery. Some authors regard them as precursors of squamous cell carcinomas and not as actual carci- nomas. There is, however, agreement that these lesions may progress to invasive squamous cell carcinoma - which is usually not pigmented. Both neoplasms commonly show surface scaling and commonly are devoid of pigment. Actinic keratoses are more common on the face and Bowen’s disease is more common on other body sites. Because both types are in- duced by UV-light the surrounding skin is usually typified by severe sun damaged except in cases of Bowen’s disease that are caused by human papilloma virus infection and not by UV. Pigmented variants exists for Bowen’s disease and for actinic keratose. Both are included in this set.<br>

- **vasc**<br>
    - Vascular skin lesions in the dataset range from cherry angiomas to angiokeratomas and pyogenic granulomas. Hemorrhage is also included in this category.<br>

- **df**<br>
    - Dermatofibroma is a benign skin lesion regarded as either a benign proliferation or an inflammatory reaction to minimal trauma. It is brown often showing a central zone of fibrosis dermatoscopically.<br>


### GENERATOR SETUP

In [ ]:
train_path = 'base_dir/train_dir'
valid_path = 'base_dir/val_dir'

num_train_samples = sum(list_train)
num_val_samples = sum(list_valid)
train_batch_size = 10
val_batch_size = 10
image_size = 224

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

print(num_train_samples)
print(num_val_samples)
print(train_steps)
print(val_steps)

In [ ]:

datagen = ImageDataGenerator(
    preprocessing_function= \
    tf.keras.applications.mobilenet.preprocess_input)

train_batches = datagen.flow_from_directory(train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=train_batch_size)

valid_batches = datagen.flow_from_directory(valid_path,
                                            target_size=(image_size,image_size),
                                            batch_size=val_batch_size)

# Note: shuffle=False causes the test dataset to not be shuffled
test_batches = datagen.flow_from_directory(valid_path,
                                            target_size=(image_size,image_size),
                                            batch_size=1,
                                            shuffle=False)

### INITIALISING MODEL AS MOBILENET

In [ ]:
mobile = tf.keras.applications.mobilenet.MobileNet()

In [ ]:
mobile.summary()

In [ ]:
len(mobile.layers)

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


In [ ]:

x = mobile.layers[-6].output

x = Flatten()(x)

x = Dense(1024, activation='relu')(x)

x = Dropout(0.25)(x)
predictions = Dense(9, activation='softmax')(x)

model = Model(inputs=mobile.input, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
for layer in model.layers[:-88]:
    layer.trainable = False

### TRAINING

In [ ]:
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

In [ ]:
model.compile(Adam(learning_rate=0.01), loss='categorical_crossentropy', 
              metrics=[categorical_accuracy, top_2_accuracy, top_3_accuracy])



In [ ]:
print(valid_batches.class_indices)

In [ ]:
class_weights={
    0: 12-(10*list_train[0]/sum(list_train)), # akiec
    1: 12-(10*list_train[1]/sum(list_train)), # bcc
    2: 12-(10*list_train[2]/sum(list_train)), # bkl
    3: 12-(10*list_train[3]/sum(list_train)), # df
    4: 10-(10*list_train[4]/sum(list_train)), #face
    5: 15-(10*list_train[5]/sum(list_train)), # mel 
    6: 15-(10*list_train[6]/sum(list_train)), #normal
    7: 7-(10*list_train[7]/sum(list_train)), # nv
    8: 10-(10*list_train[8]/sum(list_train)), # vasc
}

print(class_weights)
print(list_train)

In [ ]:

filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_top_3_accuracy', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=2, 
                                   verbose=1, mode='max', min_lr=0.00001)
                              
                              
callbacks_list = [checkpoint, reduce_lr]

history = model.fit(train_batches, steps_per_epoch=train_steps, 
                              class_weight=class_weights,
                    validation_data=valid_batches,
                    validation_steps=val_steps,
                    epochs=20, verbose=1,
                   callbacks=callbacks_list)


### Evaluate the model using the val set

In [ ]:
# get the metric names so we can use evaulate_generator
model.metrics_names

In [ ]:
# Here the the last epoch will be used.

val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = \
model.evaluate_generator(test_batches, 
                        steps=sum(list_valid))

print('val_loss:', val_loss)
print('val_cat_acc:', val_cat_acc)
print('val_top_2_acc:', val_top_2_acc)
print('val_top_3_acc:', val_top_3_acc)

In [ ]:
# Here the best epoch will be used.

model.load_weights('model.h5')

val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = \
model.evaluate_generator(test_batches, 
                        steps=sum(list_valid))

print('val_loss:', val_loss)
print('val_cat_acc:', val_cat_acc)
print('val_top_2_acc:', val_top_2_acc)
print('val_top_3_acc:', val_top_3_acc)

### Plot the Training Curves

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
train_top2_acc = history.history['top_2_accuracy']
val_top2_acc = history.history['val_top_2_accuracy']
train_top3_acc = history.history['top_3_accuracy']
val_top3_acc = history.history['val_top_3_accuracy']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, acc, 'bo', label='Training cat acc')
plt.plot(epochs, val_acc, 'b', label='Validation cat acc')
plt.title('Training and validation cat accuracy')
plt.legend()
plt.figure()


plt.plot(epochs, train_top2_acc, 'bo', label='Training top2 acc')
plt.plot(epochs, val_top2_acc, 'b', label='Validation top2 acc')
plt.title('Training and validation top2 accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, train_top3_acc, 'bo', label='Training top3 acc')
plt.plot(epochs, val_top3_acc, 'b', label='Validation top3 acc')
plt.title('Training and validation top3 accuracy')
plt.legend()


plt.show()

In [ ]:
test_labels = test_batches.classes

In [ ]:
test_batches.class_indices

In [ ]:
# make a prediction
predictions = model.predict(test_batches, steps=sum(list_valid), verbose=1)

In [ ]:
predictions.shape

### PLOT OF CONFUSION MATRIX

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()




In [ ]:
cm = confusion_matrix(test_labels, predictions.argmax(axis=1))

In [ ]:
test_batches.class_indices

In [ ]:
cm_plot_labels = ['Actinic Keratoses', 'Basal cell carcinoma', 'Benign keratosis', 'Dermatofibroma', 'Un-Zoomed Image', 'Melanoma','Normal Skin Patch','Melanocytic nevi', 'Vascular skin lesions']

plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

#### MODEL BUILD, NOW WE WILL SAVE :

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.models import model_from_json

import os

# serialize model to JSON
model_json = model.to_json()
with open("Skin_Disease.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("Skin_Disease.h5")
print("Saved model to disk")
 

### **CHALLENGES :**

- **Data Preparation:**
    - As we all know, data preparation in any ML project is the most cumbersome part.Extracting and cleaning data from different sources and integrating them is a bit tricky.
    - I prepared data from mainly 4 different sources given here :
        1. ISIC 2019
        2. HAM_10000    
        3. PH2Dataset
        4. UTK-Face

- **Classification:**
    - Skin rashes of Melanoma and Melanocytic-nevi are almost similar. Differentiating between them using even through human conscious is very difficult.

<div align="center">
<img src=
"https://user-images.githubusercontent.com/78396437/153777472-1b351907-603e-488a-8395-42e48056d5e4.png
" 
         alt="logo" 
         align="center" 
         width="400">
</div>

- **Storage:**
    - Storing such a big model also requires a lot of space.

### **SOLUTIONS:**

   - Instead of collecting normal human skin images from web. I used face images of UTK-Face Dataset and cropped it to create a custom skin dataset.

<div align="center">
<img src=
"https://user-images.githubusercontent.com/78396437/153777503-7b857955-1952-42ee-b618-39b5a2ab18bd.png" 
         alt="logo" 
         align="center" 
         width="400">
</div>


   - I used the simplest approach for training(not requiring any CSV file). I prepared subclasses in the training and validation directory on the name of my classes and directly used them for model training.

   - I removed majority of non-pigmented, subungual, ocular Melanoma which are similar to Melanocytic-nevi, to make Melanoma more differentiable.
Also we have stored the model in .json, h5 format uploaded on drive and GitHub for further use